In [ ]:
import os
from collections import defaultdict
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import sem, ttest_rel
from brainnetwork import load_data, preprocess_data, preprocess_spike_data
from brainnetwork import classify_by_timepoints, FI_by_timepoints_v2, FI_by_neuron_count
from brainnetwork import construct_correlation_network, compute_network_metrics_by_class
from brainnetwork.visualization import *
import networkx as nx


In [ ]:
base_dir = "/beegfs_hdd/data/nfs_share/users/guiyun/nishome/Micedata/"
data_path =  ["M21_1107", "M71_1024","M77_1031","M78_1017","M91_1017"]

In [ ]:
CLASS_NAMES = {1: "Convergent", 2: "Divergent", 3: "Random"}
CLASS_COLORS = {1: "#1F77B4", 2: "#D55E00", 3: "#009E73"}
CLASS_PAIRS = [(1, 2), (1, 3), (2, 3)]
NETWORK_METRICS = [
    "n_edges",
    "density",
    "mean_degree",
    "largest_component",
    "avg_clustering",
    "global_efficiency",
    "local_efficiency",
    "transitivity",
    "efficiency",
    "modularity",
    "edge_weight_mean",
    "edge_weight_abs_mean",
]
GRAPH_CONFIGS = {
    "density": {
        "label": "Top 5% edge density",
        "threshold": None,
        "top_k": 0.05,
        "weighted": False,
        "absolute": False,
    },
    "threshold": {
        "label": "Correlation >= 0.5",
        "threshold": 0.5,
        "top_k": None,
        "weighted": False,
        "absolute": False,
    },
}
STRATEGY_COLORS = {
    "density": "#4C5B5C",
    "threshold": "#E07A5F",
}
ACCURACY_COLORS = {"observed": "#22223B", "shuffled": "#B8B8B8"}


def process_mouse(mouse_id):
    data_dir = os.path.join(base_dir, mouse_id)
    print(f"Processing {mouse_id} -> {data_dir}")

    neuron_data, neuron_pos, start_edges, stimulus_data = load_data(data_dir)
    segments_spi, labels_spi, neuron_pos_spi = preprocess_spike_data(
        neuron_data,
        neuron_pos,
        start_edges,
        stimulus_data,
    )

    neuron_data_flo, neuron_pos_flo, start_edges_flo, stimulus_data_flo = load_data(
        data_dir,
        data_type="fluorescence",
    )
    segments_flo, labels_flo, neuron_pos_flo = preprocess_data(
        neuron_data_flo,
        neuron_pos_flo,
        start_edges_flo,
        stimulus_data_flo,
    )

    accuracies, time_points_acc, accuracy_std, n_folds = classify_by_timepoints(
        segments_flo,
        labels_flo,
    )
    labels_shuffled = np.random.permutation(labels_flo)
    accuracies_shuffled, time_points_shu, accuracy_std_shu, _ = classify_by_timepoints(
        segments_flo,
        labels_shuffled,
    )
    if not np.allclose(time_points_acc, time_points_shu):
        raise ValueError("Classification time axes do not match for shuffled baseline.")

    fisher_mv, time_points_fi = FI_by_timepoints_v2(
        segments_flo,
        labels_flo,
        mode="multivariate",
        reduction=None,
    )
    fisher_uv, time_points_fi_uv = FI_by_timepoints_v2(
        segments_flo,
        labels_flo,
        mode="univariate",
        reduction="mean",
    )
    if not np.allclose(time_points_fi, time_points_fi_uv):
        raise ValueError("Time axes do not match between FI variants.")

    neuron_counts, fi_values = FI_by_neuron_count(segments_flo, labels_flo)

    network_results = compute_networks_with_strategies(
        segments_spi,
        labels_spi,
        neuron_pos_spi,
    )

    return {
        "mouse_id": mouse_id,
        "fisher_mv": fisher_mv,
        "fisher_uv": fisher_uv,
        "time_points": time_points_fi,
        "network_results": network_results,
        "accuracy_curve": {
            "mouse_id": mouse_id,
            "time_axis": time_points_acc,
            "accuracies": accuracies,
            "accuracy_std": accuracy_std,
            "shuffle": accuracies_shuffled,
            "shuffle_std": accuracy_std_shu,
            "n_folds": n_folds,
        },
        "fi_by_neuron": {
            "counts": neuron_counts,
            "values": fi_values,
        },
    }


def compute_networks_with_strategies(segments, labels, neuron_pos):
    results = {}
    for strategy_name, cfg in GRAPH_CONFIGS.items():
        strategy_result = {}
        for cls in np.unique(labels):
            corr_matrix, graph, summary = construct_correlation_network(
                segments,
                labels=labels,
                class_filter=cls,
                time_range=None,
                zscore=False,
                threshold=cfg.get("threshold"),
                top_k=cfg.get("top_k"),
                weighted=cfg.get("weighted", False),
                absolute=cfg.get("absolute", False),
                balance=True,
                random_state=0,
            )
            edge_weights = extract_edge_weights(graph, corr_matrix)
            try:
                communities = nx.algorithms.community.greedy_modularity_communities(graph)
                modularity = nx.algorithms.community.modularity(graph, communities)
            except Exception:
                modularity = np.nan
            strategy_result[cls] = {
                "corr_matrix": corr_matrix,
                "corr_graph": graph,
                "summary": summary,
                "edge_weights": edge_weights,
                "edge_weight_mean": np.nanmean(edge_weights) if edge_weights.size else np.nan,
                "edge_weight_abs_mean": np.nanmean(np.abs(edge_weights)) if edge_weights.size else np.nan,
                "efficiency": summary.get("global_efficiency", np.nan),
                "modularity": modularity,
            }
        results[strategy_name] = strategy_result
    return results


def extract_edge_weights(graph, corr_matrix):
    if graph.number_of_edges() == 0:
        return np.array([], dtype=float)
    weights = []
    for u, v in graph.edges():
        weights.append(float(corr_matrix[u, v]))
    return np.asarray(weights, dtype=float)


def aggregate_curve_dict(curve_store):
    aggregated = {}
    for pair, curves in curve_store.items():
        stack = np.vstack([np.asarray(curve) for curve in curves])
        aggregated[pair] = {
            "mean": np.nanmean(stack, axis=0),
            "sem": sem(stack, axis=0, nan_policy="omit"),
            "n_mice": stack.shape[0],
        }
    return aggregated


def plot_average_fisher(curve_stats, time_points, title):
    plt.figure(figsize=(10, 4.5))
    for pair, stats_dict in sorted(curve_stats.items()):
        label = f"{CLASS_NAMES.get(pair[0], pair[0])} vs {CLASS_NAMES.get(pair[1], pair[1])}"
        mean_vals = stats_dict["mean"]
        err_vals = stats_dict["sem"]
        plt.plot(time_points, mean_vals, linewidth=2.2, label=label)
        plt.fill_between(time_points, mean_vals - err_vals, mean_vals + err_vals, alpha=0.2)
    plt.xlabel("Time (s relative to stimulus)")
    plt.ylabel("Fisher information")
    plt.title(title)
    plt.axvline(0, color="#bbbbbb", linestyle="--", linewidth=1)
    plt.grid(True, alpha=0.3)
    plt.legend(frameon=False)
    plt.tight_layout()


def _offdiag_stats(corr_matrix):
    if corr_matrix is None:
        return np.nan
    corr_matrix = np.asarray(corr_matrix)
    if corr_matrix.ndim != 2:
        return np.nan
    mask = np.triu(np.ones_like(corr_matrix, dtype=bool), k=1)
    values = corr_matrix[mask]
    values = values[np.isfinite(values)]
    return values.mean() if values.size else np.nan


def flatten_network_summary(mouse_id, strategy_name, nx_result):
    rows = []
    strategy_meta = GRAPH_CONFIGS.get(strategy_name, {})
    for cls, info in nx_result.items():
        summary = info.get("summary", {}).copy()
        row = {
            "mouse_id": mouse_id,
            "strategy": strategy_name,
            "strategy_label": strategy_meta.get("label", strategy_name),
            "class_label": cls,
            "class_name": CLASS_NAMES.get(cls, str(cls)),
        }
        for metric_name, value in summary.items():
            row[metric_name] = value
        row["efficiency"] = info.get("efficiency", np.nan)
        row["modularity"] = info.get("modularity", np.nan)
        row["edge_weight_mean"] = info.get("edge_weight_mean", np.nan)
        row["edge_weight_abs_mean"] = info.get("edge_weight_abs_mean", np.nan)
        row["mean_correlation"] = _offdiag_stats(info.get("corr_matrix"))
        rows.append(row)
    return rows


def summarize_network_metrics(df, metrics):
    summary_rows = []
    grouped = df.groupby(["strategy", "class_label"])
    for (strategy, cls), group in grouped:
        entry = {
            "strategy": strategy,
            "strategy_label": GRAPH_CONFIGS.get(strategy, {}).get("label", strategy),
            "class_label": cls,
            "class_name": CLASS_NAMES.get(cls, str(cls)),
        }
        for metric in metrics:
            if metric not in group:
                continue
            entry[f"{metric}_mean"] = group[metric].mean()
            entry[f"{metric}_sem"] = sem(group[metric], nan_policy="omit")
        summary_rows.append(entry)
    return pd.DataFrame(summary_rows).sort_values(["strategy", "class_label"])


def plot_network_metric_bars(df, metric, strategy=None):
    subset = df if strategy is None else df[df["strategy"] == strategy]
    if subset.empty or metric not in subset:
        print(f"No data available for metric {metric} (strategy={strategy}).")
        return
    pivot = subset.pivot_table(index="mouse_id", columns="class_label", values=metric)
    class_order = [cls for cls in sorted(pivot.columns) if pivot[cls].notna().any()]
    if not class_order:
        print(f"No class data for metric {metric} (strategy={strategy}).")
        return
    means, errors, labels = [], [], []
    colors = [CLASS_COLORS.get(cls, "#999999") for cls in class_order]
    for cls in class_order:
        col = pivot[cls].dropna()
        if col.empty:
            continue
        means.append(col.mean())
        errors.append(sem(col, nan_policy="omit"))
        labels.append(CLASS_NAMES.get(cls, str(cls)))
    if not means:
        print(f"Insufficient data for metric {metric} (strategy={strategy}).")
        return
    plt.figure(figsize=(6.2, 4.2))
    plt.bar(labels, means, yerr=errors, capsize=4, color=colors)
    strat_label = GRAPH_CONFIGS.get(strategy, {}).get("label", strategy)
    plt.ylabel(metric.replace("_", " ").title())
    plt.title(strat_label)
    plt.grid(True, axis="y", alpha=0.3)
    plt.tight_layout()


def aggregate_accuracy_curves(records):
    if not records:
        return {}
    time_axis = None
    observed, shuffled = [], []
    for rec in records:
        axis = np.asarray(rec["time_axis"])
        if time_axis is None:
            time_axis = axis
        elif not np.allclose(time_axis, axis):
            raise ValueError("Mismatched time axes across accuracy curves.")
        observed.append(np.asarray(rec["accuracies"]))
        shuffled.append(np.asarray(rec["shuffle"]))
    observed = np.vstack(observed)
    shuffled = np.vstack(shuffled)
    return {
        "observed": {
            "time_axis": time_axis,
            "mean": np.nanmean(observed, axis=0),
            "sem": sem(observed, axis=0, nan_policy="omit"),
            "n_mice": observed.shape[0],
        },
        "shuffled": {
            "time_axis": time_axis,
            "mean": np.nanmean(shuffled, axis=0),
            "sem": sem(shuffled, axis=0, nan_policy="omit"),
            "n_mice": shuffled.shape[0],
        },
    }


def plot_average_accuracy(accuracy_stats, title="Decoder accuracy (mean +/- SEM)"):
    if not accuracy_stats:
        print("No accuracy curves to plot.")
        return
    plt.figure(figsize=(10, 4.5))
    for key, label in (("observed", "Observed"), ("shuffled", "Label-shuffled")):
        stats_dict = accuracy_stats.get(key)
        if not stats_dict:
            continue
        color = ACCURACY_COLORS.get(key, "#666666")
        time_axis = stats_dict["time_axis"]
        mean_vals = stats_dict["mean"]
        err_vals = stats_dict["sem"]
        plt.plot(time_axis, mean_vals, label=label, color=color, linewidth=2.3)
        plt.fill_between(time_axis, mean_vals - err_vals, mean_vals + err_vals, color=color, alpha=0.18)
    plt.axvline(0, linestyle="--", color="#bbbbbb", linewidth=1.0)
    plt.xlabel("Time (s relative to stimulus)")
    plt.ylabel("Accuracy")
    plt.title(title)
    plt.grid(True, alpha=0.3)
    plt.legend(frameon=False)
    plt.tight_layout()


def paired_ttests_over_time(records):
    if not records:
        return pd.DataFrame()
    time_axis = None
    observed, shuffled = [], []
    for rec in records:
        axis = np.asarray(rec["time_axis"])
        if time_axis is None:
            time_axis = axis
        elif not np.allclose(time_axis, axis):
            raise ValueError("Mismatched time axes across accuracy curves.")
        observed.append(np.asarray(rec["accuracies"]))
        shuffled.append(np.asarray(rec["shuffle"]))
    observed = np.vstack(observed)
    shuffled = np.vstack(shuffled)
    t_stats, p_values = ttest_rel(observed, shuffled, axis=0)
    df = pd.DataFrame(
        {
            "time_point": time_axis,
            "t_stat": t_stats,
            "p_value": p_values,
            "mean_diff": np.nanmean(observed - shuffled, axis=0),
            "n_mice": observed.shape[0],
        }
    )
    return df


def aggregate_fi_by_neuron(fi_store):
    stats = []
    for count in sorted(fi_store):
        values = np.asarray(fi_store[count], dtype=float)
        if values.size == 0:
            continue
        stats.append(
            {
                "neuron_count": count,
                "mean": np.nanmean(values),
                "sem": sem(values, nan_policy="omit"),
                "n_mice": values.size,
            }
        )
    return stats


def plot_fi_by_neuron(fi_stats, title="FI vs neuron count (mean +/- SEM)"):
    if not fi_stats:
        print("No FI-by-neuron data available.")
        return
    counts = [item["neuron_count"] for item in fi_stats]
    means = [item["mean"] for item in fi_stats]
    errors = [item["sem"] for item in fi_stats]
    plt.figure(figsize=(7, 4.2))
    plt.errorbar(counts, means, yerr=errors, fmt="-o", color="#2A9D8F", capsize=4)
    plt.xlabel("Number of neurons")
    plt.ylabel("Fisher information")
    plt.title(title)
    plt.grid(True, alpha=0.3)
    plt.tight_layout()


def extract_corr_distributions(nx_result, max_abs=0.9):
    corr_data = {}
    for cls, info in nx_result.items():
        corr_matrix = np.asarray(info.get("corr_matrix"))
        if corr_matrix is None or corr_matrix.size == 0:
            continue
        mask = np.triu(np.ones_like(corr_matrix, dtype=bool), k=1)
        values = corr_matrix[mask]
        if max_abs is not None:
            values = values[np.abs(values) <= max_abs]
        corr_data[cls] = values
    return corr_data


def plot_multi_class_correlation_violin(corr_store):
    if not corr_store:
        print("No correlation distributions available.")
        return
    class_order = [cls for cls in sorted(corr_store.keys()) if corr_store[cls].size]
    if not class_order:
        print("No correlation distributions available.")
        return
    data = [corr_store[cls] for cls in class_order]
    labels = [CLASS_NAMES.get(cls, str(cls)) for cls in class_order]

    fig, ax = plt.subplots(figsize=(6.5, 4.5), dpi=200)
    parts = ax.violinplot(data, positions=np.arange(len(data)), widths=0.7, showmeans=True, showmedians=True)

    for idx, pc in enumerate(parts.get('bodies', [])):
        cls = class_order[idx]
        pc.set_facecolor(CLASS_COLORS.get(cls, "#999999"))
        pc.set_alpha(0.7)
        pc.set_edgecolor(CLASS_COLORS.get(cls, "#999999"))
        pc.set_linewidth(1.2)

    for part_name in ('cbars', 'cmins', 'cmaxes', 'cmedians', 'cmeans'):
        if part_name in parts:
            vp = parts[part_name]
            vp.set_edgecolor('#333333')
            vp.set_linewidth(1.0)

    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.set_xticks(np.arange(len(labels)))
    ax.set_xticklabels(labels)
    ax.set_ylabel('Correlation coefficient')
    ax.set_xlabel('Stimulus type')
    ax.axhline(0, color='#bbbbbb', linestyle='--', linewidth=0.8)
    ax.set_title('Pairwise correlation distributions (pooled across mice)')
    fig.tight_layout()


def summarize_edge_strengths(edge_strength_store):
    rows = []
    for strategy, class_map in edge_strength_store.items():
        strat_label = GRAPH_CONFIGS.get(strategy, {}).get("label", strategy)
        for cls, values in class_map.items():
            arr = np.asarray(values, dtype=float)
            arr = arr[np.isfinite(arr)]
            if arr.size == 0:
                continue
            rows.append(
                {
                    "strategy": strategy,
                    "strategy_label": strat_label,
                    "class_label": cls,
                    "class_name": CLASS_NAMES.get(cls, str(cls)),
                    "mean": np.nanmean(arr),
                    "sem": sem(arr, nan_policy="omit"),
                    "n_mice": arr.size,
                }
            )
    return pd.DataFrame(rows)


def plot_edge_strength_summary(edge_strength_df):
    if edge_strength_df.empty:
        print("Edge strength summary is empty.")
        return
    class_order = sorted(edge_strength_df["class_label"].unique())
    strategy_order = [name for name in GRAPH_CONFIGS.keys() if name in edge_strength_df["strategy"].unique()]
    width = 0.35 / max(len(strategy_order), 1)
    x = np.arange(len(class_order))

    plt.figure(figsize=(7, 4.2))
    for idx, strategy in enumerate(strategy_order):
        subset = edge_strength_df[edge_strength_df["strategy"] == strategy]
        means = []
        errors = []
        for cls in class_order:
            row = subset[subset["class_label"] == cls]
            if row.empty:
                means.append(np.nan)
                errors.append(np.nan)
            else:
                means.append(row["mean"].iloc[0])
                errors.append(row["sem"].iloc[0])
        offsets = x - (len(strategy_order) - 1) * width / 2 + idx * width
        color = STRATEGY_COLORS.get(strategy, None)
        plt.bar(offsets, means, width=width, label=GRAPH_CONFIGS[strategy]["label"], color=color, yerr=errors, capsize=4)
    plt.xticks(x, [CLASS_NAMES.get(cls, str(cls)) for cls in class_order])
    plt.ylabel("Mean edge weight")
    plt.title("Connection strength comparison")
    plt.grid(True, axis="y", alpha=0.3)
    plt.legend(frameon=False)
    plt.tight_layout()


def paired_ttests(df, metrics):
    records = []
    for strategy, subset in df.groupby("strategy"):
        for metric in metrics:
            pivot = subset.pivot_table(index="mouse_id", columns="class_label", values=metric)
            for cls_a, cls_b in CLASS_PAIRS:
                if cls_a not in pivot.columns or cls_b not in pivot.columns:
                    continue
                paired = pivot[[cls_a, cls_b]].dropna()
                if paired.shape[0] < 2:
                    continue
                t_stat, p_value = ttest_rel(paired[cls_a], paired[cls_b])
                records.append(
                    {
                        "strategy": strategy,
                        "strategy_label": GRAPH_CONFIGS.get(strategy, {}).get("label", strategy),
                        "metric": metric,
                        "class_a": CLASS_NAMES.get(cls_a, str(cls_a)),
                        "class_b": CLASS_NAMES.get(cls_b, str(cls_b)),
                        "n_mice": paired.shape[0],
                        "t_stat": float(t_stat),
                        "p_value": float(p_value),
                        "mean_diff": float((paired[cls_a] - paired[cls_b]).mean()),
                    }
                )
    return pd.DataFrame(records).sort_values(["metric", "p_value"])


In [ ]:
mouse_results = []
failed_mice = []

for mouse_id in data_path:
    try:
        mouse_results.append(process_mouse(mouse_id))
    except Exception as exc:
        print(f"[!] Failed to process {mouse_id}: {exc}")
        failed_mice.append({"mouse_id": mouse_id, "error": repr(exc)})

print(f"Processed {len(mouse_results)} mice (failed: {len(failed_mice)}).")
failed_mice


In [ ]:
fisher_mv_store = defaultdict(list)
fisher_uv_store = defaultdict(list)
network_rows = []
accuracy_records = []
fi_by_neuron_store = defaultdict(list)
corr_distribution_store = defaultdict(list)
time_axis = None

for result in mouse_results:
    if time_axis is None:
        time_axis = result["time_points"]
    elif not np.allclose(time_axis, result["time_points"]):
        raise ValueError("Mismatched FI time axes across mice.")

    for pair, curve in result["fisher_mv"].items():
        fisher_mv_store[pair].append(np.asarray(curve))
    for pair, curve in result["fisher_uv"].items():
        fisher_uv_store[pair].append(np.asarray(curve))

    network_rows.extend(flatten_network_summary(result["mouse_id"], result["nx_result"]))

    accuracy_payload = result.get("accuracy_curve")
    if accuracy_payload:
        accuracy_records.append(accuracy_payload)

    fi_payload = result.get("fi_by_neuron")
    if fi_payload:
        for count, value in zip(fi_payload["counts"], fi_payload["values"]):
            fi_by_neuron_store[int(count)].append(float(value))

    corr_payload = extract_corr_distributions(result["nx_result"])
    for cls, values in corr_payload.items():
        if values.size:
            corr_distribution_store[cls].append(values)

print(
    f"Collected {len(fisher_mv_store)} Fisher pairs, "
    f"{len(accuracy_records)} accuracy curves, "
    f"{sum(len(v) for v in fi_by_neuron_store.values())} FI-by-neuron entries, "
    f"and {len(network_rows)} network rows."
)


In [ ]:
if time_axis is not None and fisher_mv_store:
    fisher_mv_stats = aggregate_curve_dict(fisher_mv_store)
    plot_average_fisher(fisher_mv_stats, time_axis, "Multivariate Fisher information (mean +/- SEM)")
else:
    print("Multivariate Fisher information curves are not available.")

if time_axis is not None and fisher_uv_store:
    fisher_uv_stats = aggregate_curve_dict(fisher_uv_store)
    plot_average_fisher(fisher_uv_stats, time_axis, "Univariate Fisher information (mean +/- SEM)")
else:
    print("Univariate Fisher information curves are not available.")


In [ ]:
if accuracy_records:
    accuracy_stats = aggregate_accuracy_curves(accuracy_records)
    plot_average_accuracy(accuracy_stats)
else:
    print("Classification accuracy curves are not available.")


In [ ]:
if accuracy_records:
    accuracy_ttest_df = paired_ttests_over_time(accuracy_records)
    accuracy_ttest_df
else:
    accuracy_ttest_df = pd.DataFrame()
    print("No decoder accuracy data for t-tests.")


In [ ]:
if fi_by_neuron_store:
    fi_by_neuron_stats = aggregate_fi_by_neuron(fi_by_neuron_store)
    plot_fi_by_neuron(fi_by_neuron_stats)
    pd.DataFrame(fi_by_neuron_stats)
else:
    print("FI-by-neuron curves are not available.")


In [ ]:
if corr_distribution_store:
    plot_multi_class_correlation_violin(corr_distribution_store)
else:
    print("No correlation distributions aggregated across mice.")


In [ ]:
if edge_strength_store:
    edge_strength_df = summarize_edge_strengths(edge_strength_store)
    if not edge_strength_df.empty:
        plot_edge_strength_summary(edge_strength_df)
        edge_strength_df
    else:
        print("Edge strength summary is empty.")
else:
    edge_strength_df = pd.DataFrame()
    print("No edge strength data available.")


In [ ]:
if network_rows:
    network_df = pd.DataFrame(network_rows)
    network_df
else:
    network_df = pd.DataFrame()
    print("No network metrics were collected.")


In [ ]:
if not network_df.empty:
    network_summary_df = summarize_network_metrics(network_df, NETWORK_METRICS)
    network_summary_df
else:
    network_summary_df = pd.DataFrame()
    print("Summary table is empty because network_df is empty.")


In [ ]:
if not network_df.empty:
    strategies = network_df["strategy"].dropna().unique()
    for metric in ["avg_clustering", "global_efficiency", "modularity"]:
        for strategy in strategies:
            plot_network_metric_bars(network_df, metric, strategy=strategy)
else:
    print("Skip plotting network comparisons because network_df is empty.")


In [ ]:
if not network_df.empty:
    network_ttest_df = paired_ttests(network_df, NETWORK_METRICS)
    network_ttest_df
else:
    network_ttest_df = pd.DataFrame()
    print("No paired t-tests computed because network_df is empty.")
